In [ ]:
#@title 查看显卡
!nvidia-smi

Thu Nov 30 00:45:48 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#@title 克隆代码仓库
!git clone -b v2.0.1 --single-branch https://github.com/cncbec/Bert-VITS2/

Cloning into 'Bert-VITS2'...
remote: Enumerating objects: 1706, done.
remote: Counting objects: 100% (904/904), done.
remote: Compressing objects: 100% (174/174), done.
remote: Total 1706 (delta 800), reused 753 (delta 729), pack-reused 802
Receiving objects: 100% (1706/1706), 8.07 MiB | 14.50 MiB/s, done.
Resolving deltas: 100% (1008/1008), done.


In [ ]:
#@title 安装所需要的依赖
%cd /content/Bert-VITS2/
!pip install -r requirements.txt

In [ ]:
#@title 通过直链下载必要的文件
!wget -P emotional/wav2vec2-large-robust-12-ft-emotion-msp-dim/ https://huggingface.co/audeering/wav2vec2-large-robust-12-ft-emotion-msp-dim/resolve/main/pytorch_model.bin
!wget -P bert/chinese-roberta-wwm-ext-large/ https://huggingface.co/hfl/chinese-roberta-wwm-ext-large/resolve/main/flax_model.msgpack
!wget -P bert/chinese-roberta-wwm-ext-large/ https://huggingface.co/hfl/chinese-roberta-wwm-ext-large/resolve/main/pytorch_model.bin
!wget -P bert/chinese-roberta-wwm-ext-large/ https://huggingface.co/hfl/chinese-roberta-wwm-ext-large/resolve/main/tf_model.h5
!wget -P bert/bert-base-japanese-v3/ https://huggingface.co/cl-tohoku/bert-base-japanese-v3/resolve/main/flax_model.msgpack
!wget -P bert/bert-base-japanese-v3/ https://huggingface.co/cl-tohoku/bert-base-japanese-v3/resolve/main/pytorch_model.bin
!wget -P bert/bert-base-japanese-v3/ https://huggingface.co/cl-tohoku/bert-base-japanese-v3/resolve/main/tf_model.h5
!wget -P bert/bert-base-japanese-v3/ https://huggingface.co/cl-tohoku/bert-base-japanese-v3/resolve/main/tokenizer_config.json
!wget -P bert/deberta-v3-large/ https://huggingface.co/microsoft/deberta-v3-large/resolve/main/pytorch_model.bin
!wget -P bert/deberta-v3-large/ https://huggingface.co/microsoft/deberta-v3-large/resolve/main/pytorch_model.generator.bin
!wget -P bert/deberta-v3-large/ https://huggingface.co/ku-nlp/deberta-v2-large-japanese/resolve/main/spm.model
!wget -P bert/deberta-v3-large/ https://huggingface.co/microsoft/deberta-v3-large/resolve/main/tf_model.h5
!wget -P bert/deberta-v2-large-japanese/ https://huggingface.co/ku-nlp/deberta-v2-large-japanese/resolve/main/model.safetensors
!wget -P bert/deberta-v2-large-japanese/ https://huggingface.co/ku-nlp/deberta-v2-large-japanese/resolve/main/pytorch_model.bin
!wget -P bert/deberta-v2-large-japanese/ https://huggingface.co/ku-nlp/deberta-v2-large-japanese/resolve/main/spm.model

In [ ]:
#@title 下载底模文件
#@markdown 数据集名称（**人物的英文/拼音名**，与建数据文件夹时统一；不带zip。）
DATASETNAME = "gxd"  #@param {type:"string"}
!wget -P {DATASETNAME}/models/ https://huggingface.co/Mengxing56546/Bert-VITS2_v2.0.1_base_models/resolve/main/DUR_0.pth
!wget -P {DATASETNAME}/models/ https://huggingface.co/Mengxing56546/Bert-VITS2_v2.0.1_base_models/resolve/main/D_0.pth
!wget -P {DATASETNAME}/models/ https://huggingface.co/Mengxing56546/Bert-VITS2_v2.0.1_base_models/resolve/main/G_0.pth


In [ ]:
#@title 挂载谷歌云盘
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title (有数据集文件)从谷歌云盘加载打包好的数据集进行预处理
#@markdown 压缩包路径（谷歌盘路径，传到dataset的就不改这个，没有dataset文件夹就新建一个）
ZIP_PATH = "/content/drive/MyDrive/dataset/"  #@param {type:"string"}
ZIP_NAME = ZIP_PATH + DATASETNAME
!unzip -d /content/Bert-VITS2/raw {ZIP_NAME}.zip

In [ ]:
#@title (有数据集文件)从云盘添加标注文件到实例
%cd /content
!cp /content/drive/MyDrive/dataset/clean_barbara.list Bert-VITS2/filelists
%cd ./Bert-VITS2/filelists
!mv clean_barbara.list {DATASETNAME}.list

/content
/content/Bert-VITS2/filelists


In [ ]:
#@title (没有数据集文件)把长语音以人名（英文）命名，放入data文件夹下
#比如：data/gaoxiangdong.wav

In [ ]:
!pip install -U openai-whisper

In [ ]:
#@title (没有数据集文件)切分长语音，并添加备注文件
#beizhu.py，负责给wav文件添加lab备注文件，第8行的a="gxd"，要改成跟raw文件夹的子文件（人名命名）的字符一样
!python clean_list.py

Detected language: zh
如果你要找工作,那么就要确定你拿的是100%的酬劳,否则就自己当老板。
Detected language: zh
如果一度百尺高的像素,擁有和人力一樣高的心靈,那麼它頂多長到十尺。
Detected language: zh
只要出现一个给予者,就有一个接受者。而每一个接受者的对面,都是一个给予者。
Detected language: zh
有钱人是很棒的接受者,穷人是差劲的接受者。
Detected language: zh
我比任何问题都大,我可以解决任何问题,这是有钱人的脑袋。
Detected language: zh
说到底,如果你能变成一个出力问题和克服障碍的大师,那么还有什么东西能够阻碍你成功呢?
Detected language: zh
沒有,如果什麼都擋不住你,你就所向無敵了
Detected language: zh
如果你变得所向无敌,那么你在人生中就能拥有哪些选择呢?你可以拥有全部的选择。
Detected language: zh
如果你所向无敌,那么一切都随你取用,你只需要做出选择,它就是你的。
Detected language: zh
如果你的生活中有一个大问题,这只说明一件事,你很小。
Detected language: zh
成功的秘诀就是不要逃避问题,不要在问题面前逃避。
Detected language: zh
成功的秘诀,就在于你要成长,让自己大于一切的问题。
Detected language: zh
有錢人和成功人士大於他們的問題,但是窮人和不成功的人小於他們的問題。
Detected language: zh
如果你想改变果实,你首先必须改变它的根。如果你想改变看得见的东西,你必须先改变看不见的东西。
Detected language: zh
如果你是替别人工作,领薪水,那么你永远不会有钱。
Detected language: zh
如果你想往上升到更高的層次,就必须放弃一些旧的思考和存在方式,并且接收新的方式。
Detected language: zh
有錢人就算恐懼也會採取行動窮人卻會讓恐懼擋住了他們的行動
Detected language: zh
你只在一种状态下,是真正的成长的,那就是你觉得不舒服的时候。
Detected language: zh


In [ ]:
#@title (没有数据集文件)生成对应人名的list文件
#生成filelists下的list文件，注意修改out_file = f"filelists/gxd.list"和ch_name = 'gxd'
!python slice.py

In [ ]:
#@title 重采样
%cd /content/Bert-VITS2
!python resample.py
!python resample.py

/content/Bert-VITS2
109it [00:30,  3.53it/s]
音频重采样完毕!
109it [00:30,  3.53it/s]
音频重采样完毕!


In [ ]:
#@title 执行preprocess_text.py文件，划分训练集和验证集
%cd /content/Bert-VITS2
!sed -i 's/datasetname/{DATASETNAME}/g' config.py
!python preprocess_text.py

/content/Bert-VITS2
  0% 0/89 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.642 seconds.
Prefix dict has been built successfully.
100% 89/89 [00:02<00:00, 36.10it/s]
总重复音频数：0，总未找到的音频数:0
训练集和验证集生成完成！


In [ ]:
#@title 执行bert_gen.py文件，生成.pt文件
!python bert_gen.py

In [ ]:
#@title 执行emo_gen.py文件
!python emo_gen.py


In [ ]:
#@title 已经预处理过数据集的话，就可以跳过预处理部分 直接从云盘解压处理过的数据 以及配置文件
#@markdown 从谷歌云盘加载预处理过的数据集，文件夹名和你备份的时候输入的一样
back_up_name = "gxd"  #@param {type:"string"}
BACK_UP_DATASET_PATH = "/content/drive/MyDrive/dataset/" + back_up_name
#!unzip {BACK_UP_DATASET_PATH}/dataset.zip -d /
!unzip {BACK_UP_DATASET_PATH}/ying.zip -d /
#!cp {BACK_UP_DATASET_PATH}/val.list filelists/val.list
#!cp {BACK_UP_DATASET_PATH}/train.list filelists/train.list
#!cp {BACK_UP_DATASET_PATH}/config.json configs/config.json

Archive:  /content/drive/MyDrive/dataset/ying1/ying.zip
   creating: /content/Bert-VITS2/ying/
   creating: /content/Bert-VITS2/ying/models/
  inflating: /content/Bert-VITS2/ying/models/G_2000.pth  
  inflating: /content/Bert-VITS2/ying/models/DUR_0.pth  
  inflating: /content/Bert-VITS2/ying/models/DUR_1000.pth  
  inflating: /content/Bert-VITS2/ying/models/githash  
   creating: /content/Bert-VITS2/ying/models/.ipynb_checkpoints/
  inflating: /content/Bert-VITS2/ying/models/DUR_2000.pth  
   creating: /content/Bert-VITS2/ying/models/eval/
  inflating: /content/Bert-VITS2/ying/models/eval/events.out.tfevents.1700927859.0f826d94c242.5838.1  
  inflating: /content/Bert-VITS2/ying/models/D_2000.pth  
  inflating: /content/Bert-VITS2/ying/models/train.log  
  inflating: /content/Bert-VITS2/ying/models/events.out.tfevents.1700927859.0f826d94c242.5838.0  


In [ ]:
#@title 开始训练
#@markdown 数据集名称（**人物的英文/拼音名**，与建数据文件夹时统一；不带zip。）
DATASETNAME = "gxd"  #@param {type:"string"}
!python train_ms.py -m {DATASETNAME} -c configs/config.json

2023-11-30 01:51:43.752725: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-30 01:51:43.752788: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-30 01:51:43.752831: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-30 01:51:43.764773: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-30 01:51:45.423802: W tensorflow/c

In [ ]:
#@title 将数据集和相关文件保存到谷歌云盘的dataset文件夹下，方便下次训练使用
#压缩dataset和logs文件夹
!zip -r dataset.zip /content/Bert-VITS2/dataset
!zip -r {DATASETNAME}.zip /content/Bert-VITS2/{DATASETNAME}
#@markdown 自定义备份到谷歌云盘的dataset文件夹下的数据集文件夹名，避免混淆
dataset_name_drive = "gxd"  #@param {type:"string"}
DATASET_PATH_DRIVE = "/content/drive/MyDrive/dataset/" + dataset_name_drive
!mkdir -p {DATASET_PATH_DRIVE}

!cp /content/Bert-VITS2/dataset.zip "{DATASET_PATH_DRIVE}"
!cp /content/Bert-VITS2/{DATASETNAME}.zip "{DATASET_PATH_DRIVE}"
!cp configs/config.json "{DATASET_PATH_DRIVE}"
!cp filelists/train.list "{DATASET_PATH_DRIVE}"
!cp filelists/val.list "{DATASET_PATH_DRIVE}"

  adding: content/Bert-VITS2/dataset/ (stored 0%)
  adding: content/Bert-VITS2/dataset/ying/ (stored 0%)
  adding: content/Bert-VITS2/dataset/ying/ying_83.bert.pt (deflated 75%)
  adding: content/Bert-VITS2/dataset/ying/ying_130.bert.pt (deflated 75%)
  adding: content/Bert-VITS2/dataset/ying/ying_103.emo.npy (deflated 7%)
  adding: content/Bert-VITS2/dataset/ying/ying_117.bert.pt (deflated 75%)
  adding: content/Bert-VITS2/dataset/ying/ying_42.wav (deflated 23%)
  adding: content/Bert-VITS2/dataset/ying/ying_139.spec.pt (deflated 8%)
  adding: content/Bert-VITS2/dataset/ying/ying_159.spec.pt (deflated 8%)
  adding: content/Bert-VITS2/dataset/ying/ying_125.emo.npy (deflated 7%)
  adding: content/Bert-VITS2/dataset/ying/ying_151.bert.pt (deflated 75%)
  adding: content/Bert-VITS2/dataset/ying/ying_2.emo.npy (deflated 7%)
  adding: content/Bert-VITS2/dataset/ying/ying_108.emo.npy (deflated 7%)
  adding: content/Bert-VITS2/dataset/ying/ying_167.wav (deflated 22%)
  adding: content/Bert-VI

In [ ]:
#@title 推理
#@markdown 模型名称
Model_File = "G_2000"  #@param {type:"string"}
!sed -i 's/Model_File/{Model_File}/g' config.py
!sed -i 's/datasetname/{DATASETNAME}/g' config.py
!python webui.py

| numexpr.utils | INFO | NumExpr defaulting to 2 threads.
| __main__ | INFO | Enable DEBUG-LEVEL log
/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
| utils | INFO | Loaded checkpoint './ying/models/G_2000.pth' (iteration 91)
推理页面已开启!
Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://582ba34e0b4113fc51.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
Building prefix dict from the default dictionary ...
| jieba | DEBUG | Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
| jieba | DEBUG | Loading model from cache /tmp/jieba.cache
Loading m